In [1]:
from os import walk
from joblib import Parallel, delayed
import librosa
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [2]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 24000

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "mfcc": [],
    "labels": []
}

In [3]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=True)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "mfcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        # sample = Audio.normalize(sample)

        # mfcc = Audio.mfcc(sample, rate=sr)
        mfcc = librosa.feature.mfcc(sample, sr=sr, n_mfcc=13, hop_length=512, n_fft=2048, lifter=22)

        # mfcc = mfcc.T

        m['mfcc'].append(mfcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [4]:
def object_mfcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['mfcc'].extend(i['mfcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/mfcc/mfcc_{sampling_rate}.json', data)


In [5]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if i is not None)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    7.4s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    8.9s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    9.5s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   12.0s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:   12.8s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:   13.2s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   14.8s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   14.9s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:   15.5s
[Parallel(

In [6]:
    object_mfcc_to_json(m)

In [7]:
len(data['labels'])

17669